In [32]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
import torchvision.transforms.functional as TF

/Users/aneeshsathe/miniconda3/envs/torch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/aneeshsathe/miniconda3/envs/torch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <47DCCE5B-5493-3979-96BF-5E3CEE06843B> /Users/aneeshsathe/miniconda3/envs/torch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <B5DCF36B-AC07-3EF2-A997-D38A25D2CCE2> /Users/aneeshsathe/miniconda3/envs/torch/lib/python3.8/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [19]:
class doubleConvolution(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(doubleConvolution, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x

In [33]:
class UNet(nn.Module):
    def __init__(self, in_channels = 3, out_channels = 3, features = [64, 128, 256, 512]):
        super(UNet, self).__init__()
        self.up_conv = nn.ModuleList()
        self.down_conv = nn.ModuleList()
        self.pool = nn.MaxPool2d(stride = 2, kernel_size = 2)
        
        for feature in features:
            self.down_conv.append(doubleConvolution(in_channels, feature))
            in_channels = feature
        
        for feature in reversed(features):
            self.up_conv.append(nn.ConvTranspose2d(feature*2, feature, kernel_size = 2, stride = 2))
            self.up_conv.append(doubleConvolution(feature*2, feature))
        
        self.bottomConv = doubleConvolution(features[-1], features[-1]*2)
        
        self.lastConv = nn.Conv2d(features[0], out_channels, kernel_size = 1)
        
    def forward(self, x):
        skip_connections = []
        for down in self.down_conv:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottomConv(x)
        skip_connections = skip_connections[::-1]

        for i in range(0, len(self.up_conv), 2):
            x = self.up_conv[i](x)
            skip_connection = skip_connections[i//2]
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size = skip_connection.shape[2:])
            concat = torch.cat((skip_connection, x), dim = 1)
            x = self.up_conv[i+1](concat)

        x = self.lastConv(x)
        return x
            

In [34]:
model = UNet()
x = torch.rand([16, 3, 480, 360])
pred = model(x)
print(pred.shape)
print(x.shape)

/Users/aneeshsathe/miniconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([16, 3, 480, 360])
torch.Size([16, 3, 480, 360])
